In [1]:
import json
import pathlib
from pprint import pprint
from collections import defaultdict
from bs4 import BeautifulSoup
import requests
import re
from urllib.parse import urljoin
import sqlite3

In [2]:
pkgs_json_file = 'pkgs.json'
pkgs_db_file = 'pkgs.db'

In [3]:
with open(pkgs_json_file, 'r') as f:
    pkgs_dicts = json.load(f)

In [4]:
conn = sqlite3.connect(pkgs_db_file)
sqlite3.register_adapter(bool, int)
sqlite3.register_converter("BOOLEAN", lambda v: bool(int(v)))
c = conn.cursor()

In [5]:
c.execute(f"""
            CREATE TABLE IF NOT EXISTS 'packages'
            (pkg_name text,
             pkg_url text,
             pkg_summary text,
             installer_name text,
             installer_url text,
             anaconda_ver text,
             python_ver text,
             pkg_ver text,
             pkg_included BOOLEAN)
           """)
conn.commit()

In [6]:
c.execute(f"SELECT * FROM packages")
c.description

(('pkg_name', None, None, None, None, None, None),
 ('pkg_url', None, None, None, None, None, None),
 ('pkg_summary', None, None, None, None, None, None),
 ('installer_name', None, None, None, None, None, None),
 ('installer_url', None, None, None, None, None, None),
 ('anaconda_ver', None, None, None, None, None, None),
 ('python_ver', None, None, None, None, None, None),
 ('pkg_ver', None, None, None, None, None, None),
 ('pkg_included', None, None, None, None, None, None))

In [7]:
c.execute(f"""
            DROP TABLE IF EXISTS 'FOO'
           """)
conn.commit()

In [8]:
c.description

In [9]:
table_name = 'packages'
c.execute(f"""
           SELECT name 
           FROM sqlite_master 
           WHERE type='table' AND name='{table_name}';
           """)
c.fetchall()

[('packages',)]

In [10]:
pkgs_dicts['svn']['installers']['4.1.0_py27'].keys()

dict_keys(['installer_url', 'anaconda', 'python', 'pkg_version', 'pkg_included'])

In [11]:

for pkg in pkgs_dicts.keys():
    # pkg_url, pkg_summary, DICT installers
    for installer in pkgs_dicts[pkg]['installers'].keys():
        # installer_url, anaconda, python, pkg_version, pkg_included
        parameters = (  pkg,
                        pkgs_dicts[pkg]['pkg_url'],
                        pkgs_dicts[pkg]['pkg_summary'],
                        installer,
                        pkgs_dicts[pkg]['installers'][installer]['installer_url'],
                        pkgs_dicts[pkg]['installers'][installer]['anaconda'],
                        pkgs_dicts[pkg]['installers'][installer]['python'],
                        pkgs_dicts[pkg]['installers'][installer]['pkg_version'],
                        pkgs_dicts[pkg]['installers'][installer]['pkg_included'],
                        )
        query = "INSERT INTO packages VALUES ( ?,?,?, ?,?,?, ?,?,? )"
        c.execute(query, parameters)
conn.commit()

In [12]:

conn.close()

SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';